In [ ]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

In [ ]:
import vision as vis
import kalman
import numpy as np
import time
import cv2 as cv
import shortest_path as short
import positions as pos
import math
import motion_control as motion



COMPUTE = 0
TRAVEL = 1
LOCAL_NAV = 2
FINISH = 3

In [ ]:
# Main -> state machine
state = COMPUTE
obstacle = False
path_calculated = False

#valeurs pour initialiser
robot_pos_kalman = np.array([[0],[0],[0]]) #3-by-1 np.array
Sigma_kalman = np.diag([0.01,0.01,0.01])
HISTORY_SIZE = 15
history_kalman = [np.reshape(robot_pos_kalman,(1,3))]*HISTORY_SIZE

cap = cv.VideoCapture(2)  
cap.set(cv.CAP_PROP_AUTO_EXPOSURE, 1)
while True:
    if state == COMPUTE:
        captured, img = cap.read()
        if captured:
            cropped, cropped_img = vis.crop_map(img)
            # print(cropped_img.shape)        # uncomment to debug
            # cv.imshow("Original", img)        # uncomment to debug
            cv.imshow("Cropped", cropped_img)
            if cropped:
                im3 = np.zeros(cropped_img.shape)
                found, obstacles = vis.extract_obstacles(cropped_img)
                if found:
                    ex_obstacles = vis.expand_obstacles(obstacles)
                    cv.drawContours(im3, ex_obstacles, -1, (0, 255, 0), 3)
                    cv.imshow("expanded", im3)
                found_rob, robot_pos = pos.get_robot_position(cropped_img)
                if found_rob:
                    vis.annotate_robot(robot_pos, im3)
                found_goal, goal_pos = pos.get_goal_position(cropped_img)
                if found_goal:
                    vis.annotate_goal(goal_pos, im3)
                found_arch, arch_pos = pos.get_arch_positions(cropped_img)
                if found_arch and found_rob:
                    if math.dist(arch_pos[0],robot_pos[0])>math.dist(arch_pos[1],robot_pos[0]):
                        arch_pos = np.flip(arch_pos,0)
                    vis.annotate_arch(arch_pos, im3)
                if (found and found_rob and found_goal and found_arch):
                    if path_calculated == False:
                        vertices = vis.convert_vertice(ex_obstacles)
                        pathname = short.find_shortest_path(vertices, tuple(robot_pos[0]), tuple(arch_pos[0]))
                        path_arch =\
                        short.names_to_subpaths(pathname,vertices,tuple(robot_pos[0]), tuple(arch_pos[0]))[0]
                        vis.annotate_path(path_arch,im3)

                        pathname2 = short.find_shortest_path(vertices, tuple(arch_pos[1]), tuple(goal_pos))
                        path_goal =\
                        short.names_to_subpaths(pathname2,vertices,tuple(arch_pos[1]), tuple(goal_pos))[0]
                        vis.annotate_path(path_goal,im3)
                        cv.imshow("environment", im3)
                        
                        #initialize kalman filter
                        robot_pos_kalman = np.array([[robot_pos[0][0]],[robot_pos[0][1]],[robot_pos[2]]]) #3-by-1 np.array
                        HISTORY_SIZE = 15
                        history_kalman = [np.reshape(robot_pos_kalman,(1,3))]*HISTORY_SIZE
                        
                        state = TRAVEL
        else:
            print("There was a problem in the capture")
            break
    if state == TRAVEL:
        cv.imshow("environment", im3)
        cv.waitKey(0)
        for subgoal in path_arch[1:]:
            while not motion.check_robot_arrived(robot_pos_travel,subgoal):
                #read values from camera & sensors
                success_camera, robot_pos_camera = pos.get_robot_position(cropped_travel)
                camera_input = [robot_pos_camera, success_camera]
                await node.wait_for_variables({"motor.left.speed"})
                await node.wait_for_variables({"motor.right.speed"})
                motorspeed = [node.v.motor.right.speed, node.v.motor.left.speed]        
                Ts = 0.01 #how to check this?
                
                #get position from kalman filter
                robot_pos_kalman, Sigma_kalman, history_kalman =\
                kalman.kalmanfilter(robot_pos_kalman,Sigma_kalman,motorspeed,history_kalman, camera_input, Ts)
                robot_pos = [(robot_pos_kalman[0][0], robot_pos_kalman[1][0]), robot_pos_kalman[2][0]]
                
                #et apres, on passe robot_pos au controleur
                
                captured, img_travel = cap.read()
                cropped, cropped_travel = vis.crop_map(img_travel)
                cv.imshow("cropped",cropped_travel)
                im4 = np.zeros(cropped_travel.shape)
                vis.annotate_robot(robot_pos_travel,im4)
                vis.annotate_path(path_arch,im4)
                print("robot angle", robot_pos[1])
                err = motion.get_error(robot_pos,subgoal)
                cv.putText(im4, str(err), (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.imshow("mission status", im4)
                cv.waitKey(1)
                motor_left, motor_right = motion.speed_control(err)
                motor_left = round(motor_left)
                motor_right = round(motor_right)
                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
        for subgoal in path_goal:
            while not motion.check_robot_arrived(robot_pos,subgoal):
                #read values from camera & sensors
                success_camera, robot_pos_camera = pos.get_robot_position(cropped_travel)
                camera_input = [robot_pos_camera, success_camera]
                await node.wait_for_variables({"motor.left.speed"})
                await node.wait_for_variables({"motor.right.speed"})
                motorspeed = [node.v.motor.right.speed, node.v.motor.left.speed]        
                Ts = 0.01 #how to check this?
                
                #get position from kalman filter
                robot_pos_kalman, Sigma_kalman, history_kalman =\
                kalman.kalmanfilter(robot_pos_kalman,Sigma_kalman,motorspeed,history_kalman, camera_input, Ts)
                robot_pos = [(robot_pos_kalman[0][0], robot_pos_kalman[1][0]), robot_pos_kalman[2][0]]
                
                captured, img_travel = cap.read()
                cropped, cropped_travel = vis.crop_map(img_travel)
                cv.imshow("cropped",cropped_travel)
                im4 = np.zeros(cropped_travel.shape)
                vis.annotate_robot(robot_pos,im4)
                vis.annotate_path(path_goal,im4)
                print("robot angle", robot_pos[1])
                err = motion.get_error(robot_pos,subgoal)
                cv.putText(im4, str(err), (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.imshow("mission status", im4)
                cv.waitKey(1)
                motor_left, motor_right = motion.speed_control(err)
                motor_left = round(motor_left)
                motor_right = round(motor_right)
                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
        state = FINISH
    if state == FINISH:
        v = {
                    "motor.left.target": [0],
                    "motor.right.target": [0],
                }
        aw(node.set_variables(v))
            #print("motor_left",motor_left)
            #print("motor_right", motor_right)
            #motor_left_target = int(motor_left)
            #motor_right_target = int(motor_right)
            #print("motor_left_t",motor_left_target)
            #print("motor_right_t", motor_right_target)          
    cv.imshow("environment",im3)
    #cv.waitKey(0)
cap.release()
print("finished!")
# cv.destroyAllWindows()
    

"""
# robot run    
elif short.find_shortest_path() == True:
    state = 1

obstacle = check_obstacle(prox_horizontal)
    
        # global navigation
        if  obstacle == False:
            motion_control

        # local navigation
        elif obstacle == True:
"""
            
            
        
    

In [ ]:
OBSTACLE_THRESHOLD 10

def check_obstacle(prox_horizontal):
    prox_values = [prox_horizontal[0], prox_horizontal[4]]
    if prox_values[0] > OBSTACLE_THRESHOLD or prox_values[1] > OBSTACLE_THRESHOLD:
        obstacle = True
        
    else:
        obstacle = False
        
        
    